# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https:/

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/deepseek-ai/DeepSeek-R1',
 '/deepseek-ai/DeepSeek-R1-Distill-Qwen-32B',
 '/deepseek-ai/DeepSeek-R1-Zero',
 '/hexgrad/Kokoro-82M',
 '/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',
 '/models',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/hexgrad/Kokoro-TTS',
 '/spaces/webml-community/deepseek-r1-webgpu',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/cais/hle',
 '/datasets/bespokelabs/Bespoke-Stratos-17k',
 '/datasets/HumanLLMs/Human-Like-DPO-Dataset',
 '/datasets/yale-nlp/MMVU',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'contact page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
deepsee

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/brand'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1\nUpdated\nabout 18 hours ago\n•\n131k\n•\n2.93k\ndeepseek-ai/DeepSeek-R1-Distill-Qwen-32B\nUpdated\nabout 18 hours ago\n•\n98.1k\n•\n518\ndeepseek-ai/DeepSeek-R1-Zero\nUpdated\nabout 18 hours ago\n•\n6k\n•\n464\nhexgrad/Kokoro-82M\nUpdated\n2 days ago\n•\n37.5k\n•\n2.43k\ndeepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B\nUpdated\nabout 18 hours ago\n•\n107k\n•\n410\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n52

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'contact page', 'url': 'https://huggingface.com/contact'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Company Brochure

---

## Welcome to Hugging Face

### The AI Community Building the Future

Hugging Face is at the forefront of the artificial intelligence revolution, empowering the machine learning community through collaboration on models, datasets, and applications. With over **400k+ models** and **100k+ datasets**, we provide a platform where data scientists, developers, and researchers can explore, share, and develop machine learning tools seamlessly.

---

## Our Offerings

- **Models**: Access an extensive library of machine learning models actively updated by the community.
- **Datasets**: Collaborate and leverage countless datasets to enhance your projects.
- **Spaces**: Create and showcase applications with diverse modalities including text, image, video, audio, and even 3D.
- **Enterprise Solutions**: Tailored services for organizations seeking enterprise-grade security, dedicated support, and scalability for their AI solutions.

**Popular Customers**:
- **Ai2**
- **Meta**
- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Grammarly**

---

## Company Culture

At Hugging Face, we believe in the power of community and collaboration. Our culture emphasizes:
- **Openness**: We promote an open-source environment, allowing contributions from individuals and organizations alike.
- **Innovation**: Constantly pushing the boundaries of what’s possible in AI, with cutting-edge tools like **Transformers** and **Diffusers** at the core of our mission.
- **Support**: Providing substantial support to our users and being a reliable partner in their AI journey.

---

## Careers at Hugging Face

We are always on the lookout for talented and passionate individuals to join our growing team. If you have a love for AI, machine learning, or cutting-edge technology, check our [Careers Page](#) for current openings. Whether you're a software engineer, data scientist, product manager, or in support roles, there is a place for you at Hugging Face.

**Why Work With Us?**
- Engage in impactful projects with a community-driven ethos.
- Access to cutting-edge resources and ongoing learning opportunities.
- A welcoming and inclusive atmosphere that champions diversity.

---

## Join Us!

Explore our platform, collaborate on compelling machine learning applications, and be part of a transformative community. Sign up today to unlock a world of possibilities at [Hugging Face](https://huggingface.co).

---

**Follow Us**:  
[GitHub](https://github.com/huggingface) | [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://linkedin.com/company/huggingface) | [Discord](https://discord.com/invite/huggingface)

---

### Together, Let’s Build the Future of AI!

---

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

---

## About Us
**Hugging Face** is not just a company, but a vibrant community at the forefront of artificial intelligence and machine learning. Our mission is simple yet profound: to build the future of AI through collaboration. We provide a versatile platform where machine learning enthusiasts, researchers, and industry professionals engage on models, datasets, and applications.

---

## Our Offerings

### Collaborate and Innovate
- **Models**: Access our extensive library with over **400k models** spanning various applications, designed to meet the needs of the ML community.
- **Datasets**: Explore a collection of **100k+ datasets** necessary for training and evaluating models across domains such as text, image, and audio.
- **Spaces**: Utilize our platform to run applications in real time, showcasing innovative projects like **Text-to-3D Generation** and multi-language Text-to-Speech models.

### Advanced Solutions
- **Compute Services**: Scale your work with our paid compute solutions, starting as low as **$0.60/hour**.
- **Enterprise Solutions**: Equip your team with enterprise-grade tools ensuring security, access control, and dedicated support starting from **$20/user/month**.

### Open Source Initiatives
We believe in building a foundation for machine learning that is accessible to everyone. Our open-source projects, such as **Transformers** and **Diffusers**, have transformed how developers and researchers work with AI.

---

## Customers
At Hugging Face, we are proud to serve over **50,000 organizations**, including industry giants like:
- Amazon Web Services
- Google
- Microsoft
- Grammarly

Our diverse clientele reflects our robust platform and our commitment to advancing AI technologies.

---

## Company Culture
Hugging Face is dedicated to fostering an inclusive and innovative workplace. Emphasizing collaboration and continuous learning, our culture thrives on shared knowledge and creativity. We believe the best ideas come from a diverse set of voices and experiences.

### Join Us
We're always on the lookout for passionate individuals to join our team. Whether you’re a seasoned ML researcher or a budding developer, we offer exciting opportunities for growth and innovation.

#### Careers
Explore open positions and become a part of a team that's shaping the future of machine learning: [Check out our Careers Page](#)

---

## Contact Us
To learn more about our offerings or if you have any inquiries, visit our website or reach out through our social media channels:

- [GitHub](#)
- [Twitter](#)
- [LinkedIn](#)
- [Discord](#)

Together, let’s build the future of AI. Join the Hugging Face community today!

--- 

*Hugging Face - The AI community building the future.*

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Company Brochure

### Overview
**Hugging Face** is a pioneering platform in the AI community focused on building the future of artificial intelligence and machine learning technologies. Our collaborative community provides a space for developers, researchers, and enthusiasts to create, discover, and improve machine learning models, datasets, and applications.

### Mission
As the **home of machine learning**, Hugging Face is dedicated to accelerating innovation in AI. We strive to empower the machine learning community to harness the full potential of AI technologies, with a strong emphasis on collaboration and open-source development.

### Our Offerings
- **Models:** Access and contribute to over **400k models** including cutting-edge solutions for text, image, and audio processing.
- **Datasets:** Explore and curate more than **100k datasets** for various AI and ML tasks.
- **Spaces:** A platform feature enabling users to run applications and showcase their work, fostering the sharing of ideas and solutions.
- **Enterprise Solutions:** Our paid compute and enterprise offerings ensure organizations have access to enterprise-grade security and dedicated support for robust AI deployments.

### Who We Serve
Hugging Face is trusted by over **50,000 organizations** across the globe, including industry leaders such as:
- Meta
- Amazon Web Services
- Google
- Microsoft
- Intel
- Grammarly

### Company Culture
We cultivate a collaborative, inclusive, and innovative environment where every team member is valued. Our community-driven approach emphasizes open-source collaboration, contributing to collective success in the AI landscape. We encourage learning, sharing, and pioneering new ideas, making Hugging Face a vibrant place for creativity and growth.

### Careers at Hugging Face
At Hugging Face, we're always on the lookout for passionate individuals to join our team. We offer a dynamic work environment where employees can thrive professionally and personally. Our job opportunities span various roles, from engineering and data science to marketing and community management. Join us in our mission to shape the future of AI!

### Join Us
Explore more about Hugging Face at [huggingface.co](https://huggingface.co) to learn about our community, explore job openings, and discover how you can contribute to the breakthrough developments in AI.

---

Hugging Face – The AI community building the future.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>